# Pytorch Net 初体验


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## 定义net

In [4]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net,self).__init__()
        ## 1个input channel,6个output channels,5*5的卷积
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)

        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)


Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


输出:

Net(

  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))

  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))

  (fc1): Linear(in_features=400, out_features=120, bias=True)

  (fc2): Linear(in_features=120, out_features=84, bias=True)

  (fc3): Linear(in_features=84, out_features=10, bias=True)
  
)

你刚定义了一个前馈函数,然后反向传播函数被自动通过autograd定义了,你可以使用任何张量操作在前馈函数上.
一个模型的训练参数可以通过调用net.parameters()返回:

In [5]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


让我们尝试随机生成一个32*32的输入.注意:期望输入的维度是32*32,为了使整个网络在MNIST数据集上,你需要把数据集图片的维度修改为32*32

In [6]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0624, -0.0583, -0.0969,  0.0790,  0.1254, -0.1156, -0.0258, -0.0068,
          0.1102,  0.0970]], grad_fn=<AddmmBackward0>)


把所有梯度缓存器置零,用随机的梯度来反向传播

In [7]:
net.zero_grad()
out.backward(torch.randn(1, 10))

在此,我们完成了:
* 定义一个神经网络
* 处理输入及调用反向传播

还剩下:
* 计算损失值
* 更新网络中的权重

一个损失函数要有一对输入:模型目标和输出,然后计算一个值来评估输出距离目标有多远.

有一些不同的损失函数在nn包中.一个简单的损失函数就是nn.MSELoss,这个计算了均方误差.

例如:

In [8]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()
loss = criterion(output,target)
print(loss)

tensor(1.2067, grad_fn=<MseLossBackward0>)


现在,如果你跟随损失到反向传播路径,可以使用它的.grad_fn属性,你将会看到这样一个计算图:

input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d

-> view -> linear -> relu -> linear -> relu -> linear

-> MSELoss

-> loss

所以,当我们调用loss.backward()时,整个图都会微分,并且所有在图中的require_grd=True的张量将会让他们的grad张量累计梯度.

为了演示,我们将跟随以下步骤来反向传播.



In [9]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

为了实现反向传播损失,我们所有要做的事情仅仅是使用loss.backward(),你要清空现存的梯度,要不然将会和现存的梯度累计到一起.

现在我们调用loss.backward(),然后看下con1的偏置项在反向传播之前和之后的变化.

In [10]:
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)
loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0110,  0.0091, -0.0003, -0.0121, -0.0047,  0.0074])
